## Naive Bayes

Naive bayes es un clasificador simple conocido por funcionar bien cuando solo hay un pequeño número de observaciones disponibles.Como ejemplo rearemos un clasificador de Naive Bayes normal desde el principio y lo usaremos para predecir la clase de un punto de datos previamente no visto.

In [1]:
# Librerias
import pandas as pd
import numpy as np

In [2]:
# Creamos datos
data = pd.DataFrame()

# Variable objetivo
data['Genero'] = ['M','M','M','M','F','F','F','F']

# Variable de caracteristicas
data['Altura'] = [6,5.92,5.58,5.92,5,5.5,5.42,5.75]
data['Peso'] = [180,190,170,165,100,150,130,150]
data['Talla_zapato'] = [12,11,12,10,6,8,7,9]
data

,Genero,Altura,Peso,Talla_zapato
0,M,6.00,180,12
1,M,5.92,190,11
2,M,5.58,170,12
3,M,5.92,165,10
4,F,5.00,100,6
5,F,5.50,150,8
6,F,5.42,130,7
7,F,5.75,150,9


El conjunto de datos anterior se usa para construir nuestro clasificador. A continuación creamos una nueva persona para la que conocemos sus valores de características, pero no su género. Nuestro objetivo es predecir su género.

In [3]:
persona = pd.DataFrame()

# Variables caracteristicas para esta persona(fila)
persona['Altura'] = [6]
persona['Peso'] = [130]
persona['Talla_zapato'] = [8]

persona

,Altura,Peso,Talla_zapato
0,6,130,8


### Teorema de Bayes

Es un famoso resultado, que permite hacer predicciones de la data. 

$$p(A|B) = \frac{p(B|A)p(A)}{p(B)}$$

En un clasificador bayesiano, estamos interesados en conocer la clase (por ejemplo, hombre o mujer, spam o ham) de una observación dada la información:

$$p(clase\mid\textbf{data}) = \frac{p(\textbf{data}\mid clase)*p(clase)}{p(\textbf{data})}$$.

Donde:

* $\text{clase} $ es una clase particular (por ejemplo, masculino)
* $\mathbf{\text{data}} $ es un dato de observación
* $p(\text{clase}\mid \mathbf{\text{data}}) $ se llama posterior
* $p(\mathbf{\text{data}} \mid {clase})$ se llama la probabilidad o likelihood
* $p(\text{clase})$ se llama prior
* $p(\mathbf{\text{data}})$ se llama la probabilidad marginal

En un clasificador bayesiano, calculamos el posterior (técnicamente solo calculamos el numerador del posterior) para cada clase de cada observación. Luego, clasificamos la observación según la clase con el valor posterior más grande. En nuestro ejemplo, tenemos una observación para predecir y dos clases posibles (M y F), por lo tanto, calcularemos dos posteriores: uno para M y uno para F.


$$p(persona\ es\ femenina\ |\ \mathbf{\text{datos de personas}}) = \frac{p(\mathbf{\text{datos de personas}}\ |\ persona\ es\ femenina\ )}{p(\mathbf{\text{datos de personas}})}$$

### Clasificador Naive Bayes normal

Un clasificador Bayesiano normal es probablemente el tipo más popular de clasificador de bayes. Aprendamos que significa y como se usa este clasificador, con las ecuaciones de bayes para nuestros datos:

$$posterior(F) = \frac{p(F)*p(altura\ |\ F)*p(peso\ |\ F)*p(tam\_zapato\ |\ F)}{probabilidad\ marginal}$$



* $p(\text{F}) $ probabilidades prior. Es la probabilidad de que una observación sea F. Este es solo el número de mujeres en el conjunto de datos dividido por el número total de personas en el conjunto de datos.

* $p(altura \mid F) \, p(peso \mid F) \, p(tam\_zapato \mid F)$ es la probabilidad o likelihood. Ten en cuenta que hemos desempaquetado **datos de la persona** por lo que ahora es cada característica en el conjunto de datos.

    - Si se observa cada término en el likelihood de que se observe que asumimos que cada característica no está correlacionada entre sí. Es decir, el tamaño del zapato es independiente del peso o la altura.  Esto obviamente no es cierto, y es una suposición "ingenua", de ahí el nombre "naive bayes".

    - En segundo lugar, suponemos que el valor de las características (por ejemplo, la altura de las mujeres, el peso de las mujeres) se distribuye de manera normal o Gaussiana.
    
* **probabilidad marginal**, es probablemente una de las partes más confusas de los enfoques bayesianos. En ejemplos elementales es completamente posible calcular la probabilidad marginal. Sin embargo, en muchos casos, es extremadamente difícil o imposible encontrar el valor de la probabilidad marginal. Esto no es tan problemático para nuestro clasificador como podría pensar. ¿Por qué? 
    Por que  no nos importa cuál es el verdadero valor posterior, solo nos importa qué clase tiene el valor posterior más alto y  debido a que la probabilidad marginal es la misma para todas las clases 1) podemos ignorar el denominador, 2) calcular solo el numerador posterior para cada clase, y 3) elegir el numerador más grande. Es decir, podemos ignorar el denominador posterior y hacer una predicción únicamente sobre los valores relativos del numerador posterior.


### Calculando priors 

In [4]:
# Numero de M
n_m = data['Genero'][data['Genero'] == 'M'].count()

# Numero de F
n_f = data['Genero'][data['Genero'] == 'F'].count()

# Total filas
total_ppl = data['Genero'].count()

In [5]:
# Numero de M divididos por el numero de filas
P_m = n_m/total_ppl

# Numero de F divididos por el numero de filas
P_f = n_f/total_ppl

### Calculando likelihood

Hay que recordar que cada término  es un pdf normal. Esto significa que para cada combinación de clase (por ejemplo, F) y una característica (por ejemplo, altura), necesitamos calcular la varianza y el valor medio de los datos. Pandas lo hace fácil:


In [6]:
# Agrupamos los datos por genero y calculamos la media para cada caracteristica
data_media = data.groupby('Genero').mean()
data_media

,Altura,Peso,Talla_zapato
Genero,,,
F,5.4175,132.50,7.50
M,5.8550,176.25,11.25


In [7]:
# Agrupamos los datos por genero y calculamos la varianza para cada caracteristica
data_varianza = data.groupby('Genero').var()
data_varianza

,Altura,Peso,Talla_zapato
Genero,,,
F,0.097225,558.333333,1.666667
M,0.035033,122.916667,0.916667


Ahora podemos crear todas las variables que necesitamos. 

In [8]:
# Medias para M
m_altura_media = data_media['Altura'][data_varianza.index == 'M'].values[0]
m_peso_media = data_media['Peso'][data_varianza.index == 'M'].values[0]
m_talla_zapato_media = data_media['Talla_zapato'][data_varianza.index == 'M'].values[0]

# Varianzas para M
m_altura_varianza = data_varianza['Altura'][data_varianza.index == 'M'].values[0]
m_peso_varianza = data_varianza['Peso'][data_varianza.index == 'M'].values[0]
m_talla_zapato_varianza = data_varianza['Talla_zapato'][data_varianza.index == 'M'].values[0]

# Medias para F
f_altura_media = data_media['Altura'][data_varianza.index == 'F'].values[0]
f_peso_media= data_media['Peso'][data_varianza.index == 'F'].values[0]
f_talla_zapato_media = data_media['Talla_zapato'][data_varianza.index == 'F'].values[0]

# Varianzas para F
f_altura_varianza= data_varianza['Altura'][data_varianza.index == 'F'].values[0]
f_peso_varianza = data_varianza['Peso'][data_varianza.index == 'F'].values[0]
f_talla_zapato_varianza = data_varianza['Talla_zapato'][data_varianza.index == 'F'].values[0]

Finalmente, necesitamos crear una función para calcular la densidad de probabilidad de cada uno de los términos del likelihood (por ejemplo, $p(\text{altura} \mid F)$.

### Funcion de probabilidad

Creamos una funcion que calcula p(x | y)

In [9]:
def p_x_dado_y(x, media_y, varianza_y):

    # Ingresamos los argumentos en un PDF 
    p = 1/(np.sqrt(2*np.pi*varianza_y)) * np.exp((-(x-media_y)**2)/(2*varianza_y))
    return p

### Clasificador Bayesiano a nuevos puntos

Podemos ignorar la probabilidad marginal (el denominador), por lo que en realidad estamos calculando es: 

$$\displaystyle{{numerador\ del\ posterior}} = {p(F) * p(altura \mid F) * p(peso \mid F) *p(tam\_zapato \mid F)}$$

Para hacer esto, solo necesitamos agregar los valores de la persona no clasificada (altura = 6), las variables del conjunto de datos (por ejemplo, la media de la altura de F) y la función (p_x_dado_y) que hicimos anteriormente:

In [10]:
# Numerador del posterior si la observacion no clasificada es M 
P_m * \
p_x_dado_y(persona['Altura'][0], m_altura_media, m_altura_varianza) * \
p_x_dado_y(persona['Peso'][0], m_peso_media, m_peso_varianza) * \
p_x_dado_y(persona['Talla_zapato'][0], m_talla_zapato_media, m_talla_zapato_varianza)

6.197071843878078e-09

In [11]:
# Numerador del posterior si la observacion no clasificada es F
P_f* \
p_x_dado_y(persona['Altura'][0], f_altura_media, f_altura_varianza) * \
p_x_dado_y(persona['Peso'][0], f_peso_media, f_peso_varianza) * \
p_x_dado_y(persona['Talla_zapato'][0], f_talla_zapato_media, f_talla_zapato_varianza)

0.0005377909183630018

Debido a que el numerador del posterior de F es mayor que M, entonces predecimos que la persona es una mujer.